# <center> ML-7. Прогнозирование биологического ответа (HW-3)
#### <center> Необходимо предсказать биологический ответ молекул (столбец *'Activity'*) по их химическому составу (столбцы *D1-D1776*).
---

[**&#8595; Скачать данные**](https://lms-cdn.skillfactory.ru/assets/courseware/v1/9f2add5bca59f8c4df927432d605fff3/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/_train_sem09__1_.zip "_train_sem09__1_.zip")

Данные представлены в формате CSV.  Каждая строка представляет молекулу. 

* Первый столбец *Activity* содержит экспериментальные данные, описывающие фактический биологический ответ [0, 1]; 
* Остальные столбцы *D1-D1776* представляют собой молекулярные **дескрипторы** — это вычисляемые свойства, которые могут фиксировать некоторые характеристики молекулы, например размер, форму или состав элементов.


Предварительная обработка не требуется, данные уже закодированы и нормализованы.

В качестве метрики будем использовать **F1-score**.

Необходимо обучить две модели: **логистическую регрессию** и **случайный лес**. Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. Важно использовать **все четыре метода** (*GridSeachCV*, *RandomizedSearchCV*, *Hyperopt*, *Optuna*) хотя бы по разу, максимальное количество итераций не должно превышать 50.

---

### Baseline

##### Предобработка

In [48]:
import numpy as np
import pandas as pd

from sklearn import model_selection
from sklearn import linear_model
from sklearn import tree
from sklearn import ensemble
from sklearn import metrics

In [49]:
data = pd.read_csv('data/_train_sem09 (1).csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [50]:
data['Activity'].value_counts(True)

Activity
1    0.542255
0    0.457745
Name: proportion, dtype: float64

Дисбаланс классов небольшой. Но стратификацию всё равно лучше оставить.

In [51]:
X = data.drop('Activity', axis=1)
y = data['Activity']
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, stratify=y, random_state = 42, test_size = 0.2)

##### Обучение с параметрами по умолчанию

Сначала обучим модели с параметрами по умолчанию кроме random_state и max_iter.

LogisticRegression

In [52]:
log_reg = linear_model.LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(X_train, y_train)
y_test_pred_lr = log_reg.predict(X_test)
print('LogisticRegression test f1-score: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_lr)))

LogisticRegression test f1-score: 0.78


RandomForestClassifier

In [53]:
rf_clf = ensemble.RandomForestClassifier(random_state=42)
rf_clf.fit(X_train, y_train)
y_test_pred_rf = rf_clf.predict(X_test)
print('RandomForestClassifier test f1-score: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_rf)))

RandomForestClassifier test f1-score: 0.80


Видно что обе модели даже с параметрами по умолчанию дают хорошие значения метрики. Случайный лес чуть лучше.

---
### Подбор гиперпараметров

##### GridSearchCV

LogisticRegression

In [54]:
# Задаём сетку параметров
# Т.к. разные алгоритмы поддерживают разные типы регуляризации, зададим два словаря
param_grid_lr = [
              {'penalty': ['l2', 'none'],
                'C': list(np.linspace(0.01, 1, 10, dtype=float)),
               'solver': ['lbfgs', 'saga']},
              
              {'penalty': ['l1', 'l2'],
                'C': list(np.linspace(0.01, 1, 10, dtype=float)),
               'solver': ['liblinear']}
             ]

In [55]:
grid_search_lr = model_selection.GridSearchCV(
    # Модель
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=1000), 
    # Сетка параметров
    param_grid=param_grid_lr, 
    # Количество фолдов для кросс-валидации
    cv=5,
    # Все доступные ядра
    n_jobs = -1
)

In [56]:
grid_search_lr.fit(X_train, y_train)
# Выводим лучшие значения гиперпараметров
print("Best parameters: {}".format(grid_search_lr.best_params_))
# Считаем f1-меру
y_test_pred_lr_gs = grid_search_lr.predict(X_test)
print('LogisticRegression GridSearchCV test f1-score: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_lr_gs)))

Best parameters: {'C': 0.45, 'penalty': 'l1', 'solver': 'liblinear'}
LogisticRegression GridSearchCV test f1-score: 0.78


RandomForestClassifier

In [57]:
param_grid_rf = [
              {'n_estimators': list(np.linspace(100, 500, 5, dtype=int)),
                'criterion': ['gini', 'entropy'],
               'max_depth': list(np.linspace(5, 30, 6, dtype=int)),
               'min_samples_leaf': list(np.linspace(1, 30, 5, dtype=int))}
             ]

In [58]:
grid_search_rf = model_selection.GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_grid=param_grid_rf, 
    cv=5,
    n_jobs = -1
)

In [59]:
grid_search_rf.fit(X_train, y_train)
print("Best parameters: {}".format(grid_search_rf.best_params_))

y_test_pred_rf_gs = grid_search_rf.predict(X_test)
print('RandomForestClassifier GridSearchCV test f1-score: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_rf_gs)))

Best parameters: {'criterion': 'gini', 'max_depth': 15, 'min_samples_leaf': 1, 'n_estimators': 200}
RandomForestClassifier GridSearchCV test f1-score: 0.81


Как видно, метрика на логистической регресси не изменилась, а на случайном лесе подросла незначительно. При этом время работы существенно вазросло.

\----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

##### RandomizedSearchCV

Будем использовать те же сетки параметров, что и в GridSearchCV.

LogisticRegression

In [60]:
param_distributions_lr = param_grid_lr

random_search_lr = model_selection.RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=1000), 
    param_distributions=param_distributions_lr, 
    cv=5,
    # В GridSearchCV было 60 комбинаций, попробуем оставить половину
    n_iter = 30, 
    n_jobs = -1
)
# Выводим значения оптимальных параметров и метрику
random_search_lr.fit(X_train, y_train)
print("Best parameters: {}".format(random_search_lr.best_params_))

y_test_pred_lr_rs = random_search_lr.predict(X_test)
print('LogisticRegression RandomizedSearchCV test f1-score: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_lr_rs)))

Best parameters: {'solver': 'liblinear', 'penalty': 'l1', 'C': 0.45}
LogisticRegression RandomizedSearchCV test f1-score: 0.78


Метрика и набор параметров такие же, как в GridSearchCV, зато время существенно сократилось.

RandomForestClassifier

In [61]:
param_distributions_rf = param_grid_rf

random_search_rf = model_selection.RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions_rf, 
    cv=5,
    # Количество итераций поставим максимальное из возможных по условию
    n_iter = 50, 
    n_jobs = -1
)
# Выводим значения оптимальных параметров и метрику
random_search_rf.fit(X_train, y_train)
print("Best parameters: {}".format(random_search_rf.best_params_))

y_test_pred_rf_rs = random_search_rf.predict(X_test)
print('LogisticRegression RandomizedSearchCV test f1-score: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_rf_rs)))

Best parameters: {'n_estimators': 300, 'min_samples_leaf': 1, 'max_depth': 25, 'criterion': 'gini'}
LogisticRegression RandomizedSearchCV test f1-score: 0.80


Метрика немного просела, видимо случайным образом не была выбрана оптимальная комбинация параметров, но время сократилось в разы.

\----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

##### Hyperopt